In [442]:
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from tqdm import tqdm

In [227]:
maestro_consultora = pd.read_csv('maestro_consultora.csv').iloc[:, 1:]
maestro_consultora.head()

,campanaingreso,campanaultimopedido,IdConsultora,estadocivil,flagsupervisor,campanaprimerpedido,flagconsultoradigital,flagcorreovalidad,edad,flagcelularvalidado
0,200907,201915,431,Casado(a),0,200907.0,0,0.0,58,1
1,199708,201914,2010,Casado(a),0,199708.0,0,NaN,58,1
2,199709,201914,3513,Casado(a),0,199709.0,0,NaN,60,1
3,201813,201911,4895,Casado(a),0,201813.0,0,NaN,50,1
4,199711,201914,6233,Casado(a),0,199711.0,0,NaN,65,1


In [466]:
edad, edad_labels = pd.qcut(maestro_consultora['edad'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'], retbins=True)
maestro_consultora['edad_cuartil'] = edad

In [478]:
consultora_filter = pd.get_dummies(maestro_consultora[
    ['IdConsultora', 'campanaingreso', 'campanaultimopedido', 'campanaprimerpedido', 'edad_cuartil']
])
consultora_filter.head()

,IdConsultora,campanaingreso,campanaultimopedido,campanaprimerpedido,edad_cuartil_Q1,edad_cuartil_Q2,edad_cuartil_Q3,edad_cuartil_Q4
0,431,200907,201915,200907.0,0,0,0,1
1,2010,199708,201914,199708.0,0,0,0,1
2,3513,199709,201914,199709.0,0,0,0,1
3,4895,201813,201911,201813.0,0,0,0,1
4,6233,199711,201914,199711.0,0,0,0,1


In [228]:
maestro_producto = pd.read_csv('maestro_producto.csv').iloc[:, 1:]
maestro_producto.head()

,idproducto,codigounidadnegocio,unidadnegocio,codigomarca,marca,codigocategoria,categoria,codigotipo,tipo
0,200056246,10,COSMETICOS,2,ESIKA,01,FRAGANCIAS,001,FRAGANCIAS/DAMAS/COLONIA Y O EAU DE TOILETE
1,200089626,10,COSMETICOS,2,ESIKA,04,CUIDADO PERSONAL,035,CUIDADO PERSONAL/TALCOS/ESPECIFICOS
2,200095496,10,COSMETICOS,2,ESIKA,01,FRAGANCIAS,001,FRAGANCIAS/DAMAS/COLONIA Y O EAU DE TOILETE
3,210089216,50,APOYO,2,ESIKA,72,PROMOCION USUARIOS,553,PROMOCION USUARIOS/HOGAR/BLOQUE MAQUILLAJE
4,200090369,10,COSMETICOS,3,CYZONE,02,MAQUILLAJE,027,MAQUILLAJE/PARA LABIOS/LAPIZ LABIAL


In [229]:
venta_campaña = pd.read_csv('dtt_fvta_cl.csv')
venta_campaña.head()

/Users/isaiashl/Desktop/env_adela/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,campana,idconsultora,codigotipooferta,descuento,ahorro,canalingresoproducto,idproducto,codigopalancapersonalizacion,palancapersonalizacion,preciocatalogo,...,realanulmnneto,realdevmnneto,realuuanuladas,realuudevueltas,realuufaltantes,realuuvendidas,realvtamnfaltneto,realvtamnneto,realvtamncatalogo,realvtamnfaltcatalogo
0,201807,544706,1,0.0,7990.0,NaN,200091158,12.0,NaN,5495.0,...,0.0,0.0,0,0,0,2,0.0,9236.0,10990.0,0.0
1,201807,638877,1,0.0,3800.0,WEB,200091762,1111.0,NaN,7690.0,...,0.0,0.0,0,0,0,1,0.0,6462.0,7690.0,0.0
2,201807,186016,1,0.0,3800.0,APP,200091762,42.0,NaN,7690.0,...,0.0,0.0,0,0,0,1,0.0,6462.0,7690.0,0.0
3,201807,579940,1,0.0,32748.0,WEB,200091762,12.0,NaN,6032.0,...,0.0,0.0,0,0,0,6,0.0,30414.0,36192.0,0.0
4,201807,342158,1,0.0,17272.0,WEB,200091762,12.0,NaN,7172.0,...,0.0,0.0,0,0,0,4,0.0,24108.0,28688.0,0.0


In [481]:
campaña_consultora['IdConsultora'].unique().shape, consultora_filter['IdConsultora'].unique().shape

((123771,), (123740,))

In [485]:
campaña_consultora = pd.read_csv('campana_consultora.csv').iloc[:,1:]
campaña_consultora = campaña_consultora.sort_values(by=['IdConsultora', 'campana'])
campaña_consultora.head()

,campana,IdConsultora,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,...,codigocanalorigen,evaluacion_nuevas,segmentacion,flagpasopedidoweb,cantidadlogueos,flagdispositivo,flagdigital,flagofertadigital,flagsuscripcion,geografia
232,201807,125,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,Est,Nivel2,0.0,NaN,0,NaN,0.0,0,13 STGO. / VI?A DEL MAR / VA
172361,201808,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,Est,Nivel2,0.0,NaN,0,NaN,0.0,0,13 STGO. / VI?A DEL MAR / VA
320095,201809,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,Est,Nivel2,0.0,NaN,0,NaN,0.0,0,13 STGO. / VI?A DEL MAR / VA
314240,201810,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,Est,Nivel2,0.0,NaN,0,NaN,0.0,0,13 STGO. / VI?A DEL MAR / VA
526321,201811,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,Est,Nivel2,0.0,NaN,0,NaN,0.0,0,13 STGO. / VI?A DEL MAR / VA


In [486]:
campaña_consultora = campaña_consultora.merge(consultora_filter[['IdConsultora', 'campanaultimopedido']], 
                                              on='IdConsultora', how='left')
campaña_consultora.dropna(subset=['campanaultimopedido'], inplace=True)

In [488]:
campaña_consultora.tail()

,campana,IdConsultora,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,...,evaluacion_nuevas,segmentacion,flagpasopedidoweb,cantidadlogueos,flagdispositivo,flagdigital,flagofertadigital,flagsuscripcion,geografia,campanaultimopedido
1501983,201906,815405,1,1,1.0,0.0,1.0,0.0,0.0,0.0,...,C_1d1,Nuevas,1.0,3.0,1,0.0,0.0,0,15 SUR GRANDE,201908.0
1501984,201906,815407,1,1,1.0,1.0,1.0,0.0,0.0,1.0,...,C_1d1,Nuevas,1.0,3.0,1,0.0,0.0,1,15 SUR GRANDE,201906.0
1501985,201906,815415,1,1,0.0,1.0,1.0,1.0,0.0,1.0,...,C_1d1,Nuevas,1.0,2.0,0,0.0,0.0,0,15 SUR GRANDE,201910.0
1501986,201906,815416,1,1,1.0,0.0,0.0,0.0,0.0,1.0,...,C_1d1,Nuevas,1.0,1.0,1,0.0,0.0,0,14 SANTIAGO / SUR CHICO,201914.0
1501987,201906,815452,1,1,1.0,0.0,1.0,1.0,0.0,0.0,...,C_1d1,Nuevas,1.0,2.0,0,0.0,0.0,0,15 SUR GRANDE,201914.0


In [489]:
campaña_consultora['diff_ultimopedido'] = campaña_consultora['campanaultimopedido'] - campaña_consultora['campana']

In [494]:
campaña_agg1 = pd.DataFrame(
    venta_campaña.groupby(
        ['idconsultora', 'campana']
    )['ahorro', 'realvtamncatalogo', 'realvtamnneto', 'realanulmnneto', 'realdevmnneto', 'realuuanuladas', 
      'realuudevueltas', 'realuufaltantes', 'realuuvendidas', 'realvtamnfaltneto'].sum()
).reset_index()

In [495]:
campaña_agg1.rename(columns={'idconsultora': 'IdConsultora'}, inplace=True)
campaña_agg1.head()

,IdConsultora,campana,ahorro,realvtamncatalogo,realvtamnneto,realanulmnneto,realdevmnneto,realuuanuladas,realuudevueltas,realuufaltantes,realuuvendidas,realvtamnfaltneto
0,202,201807,16379.0,35710.0,25022.0,0.0,0.0,0,0,0,13,0.0
1,202,201810,19302.0,57680.0,39439.0,0.0,0.0,0,0,0,17,0.0
2,202,201812,21338.0,52930.0,38142.0,0.0,0.0,0,0,0,7,0.0
3,202,201813,8522.0,45810.0,29975.0,0.0,0.0,0,0,0,10,0.0
4,202,201814,14378.0,51910.0,37335.0,0.0,0.0,0,0,0,12,0.0


In [501]:
campaña_agg1.columns = ['IdConsultora', 'campana', 'ahorro_sum', 'realvtamncatalogo_sum',
       'realvtamnneto_sum', 'realanulmnneto_sum', 'realdevmnneto_sum', 'realuuanuladas_sum',
       'realuudevueltas_sum', 'realuufaltantes_sum', 'realuuvendidas_sum',
       'realvtamnfaltneto_sum']
campaña_agg1.columns

Index(['IdConsultora', 'campana', 'ahorro_sum', 'realvtamncatalogo_sum',
       'realvtamnneto_sum', 'realanulmnneto_sum', 'realdevmnneto_sum',
       'realuuanuladas_sum', 'realuudevueltas_sum', 'realuufaltantes_sum',
       'realuuvendidas_sum', 'realvtamnfaltneto_sum'],
      dtype='object')

In [496]:
campaña_agg2 = pd.DataFrame(
    venta_campaña.groupby(
        ['idconsultora', 'campana']
    )['ahorro', 'realvtamncatalogo', 'realvtamnneto', 'realanulmnneto', 'realdevmnneto', 'realuuanuladas', 
      'realuudevueltas', 'realuufaltantes', 'realuuvendidas', 'realvtamnfaltneto'].mean()
).reset_index()

In [497]:
campaña_agg2.rename(columns={'idconsultora': 'IdConsultora'}, inplace=True)
campaña_agg2.head()

,IdConsultora,campana,ahorro,realvtamncatalogo,realvtamnneto,realanulmnneto,realdevmnneto,realuuanuladas,realuudevueltas,realuufaltantes,realuuvendidas,realvtamnfaltneto
0,202,201807,1364.916667,2975.833333,2085.166667,0.0,0.0,0.0,0.0,0.0,1.083333,0.0
1,202,201810,1206.375000,3605.000000,2464.937500,0.0,0.0,0.0,0.0,0.0,1.062500,0.0
2,202,201812,3048.285714,7561.428571,5448.857143,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
3,202,201813,946.888889,5090.000000,3330.555556,0.0,0.0,0.0,0.0,0.0,1.111111,0.0
4,202,201814,1597.555556,5767.777778,4148.333333,0.0,0.0,0.0,0.0,0.0,1.333333,0.0


In [502]:
campaña_agg2.columns = ['IdConsultora', 'campana', 'ahorro_mean', 'realvtamncatalogo_mean',
       'realvtamnneto_mean', 'realanulmnneto_mean', 'realdevmnneto_mean', 'realuuanuladas_mean',
       'realuudevueltas_mean', 'realuufaltantes_mean', 'realuuvendidas_mean',
       'realvtamnfaltneto_mean']
campaña_agg2.columns

Index(['IdConsultora', 'campana', 'ahorro_mean', 'realvtamncatalogo_mean',
       'realvtamnneto_mean', 'realanulmnneto_mean', 'realdevmnneto_mean',
       'realuuanuladas_mean', 'realuudevueltas_mean', 'realuufaltantes_mean',
       'realuuvendidas_mean', 'realvtamnfaltneto_mean'],
      dtype='object')

In [503]:
campaña_agg = campaña_agg1.merge(campaña_agg2, on=['IdConsultora', 'campana'])
campaña_agg.head()

,IdConsultora,campana,ahorro_sum,realvtamncatalogo_sum,realvtamnneto_sum,realanulmnneto_sum,realdevmnneto_sum,realuuanuladas_sum,realuudevueltas_sum,realuufaltantes_sum,...,ahorro_mean,realvtamncatalogo_mean,realvtamnneto_mean,realanulmnneto_mean,realdevmnneto_mean,realuuanuladas_mean,realuudevueltas_mean,realuufaltantes_mean,realuuvendidas_mean,realvtamnfaltneto_mean
0,202,201807,16379.0,35710.0,25022.0,0.0,0.0,0,0,0,...,1364.916667,2975.833333,2085.166667,0.0,0.0,0.0,0.0,0.0,1.083333,0.0
1,202,201810,19302.0,57680.0,39439.0,0.0,0.0,0,0,0,...,1206.375000,3605.000000,2464.937500,0.0,0.0,0.0,0.0,0.0,1.062500,0.0
2,202,201812,21338.0,52930.0,38142.0,0.0,0.0,0,0,0,...,3048.285714,7561.428571,5448.857143,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
3,202,201813,8522.0,45810.0,29975.0,0.0,0.0,0,0,0,...,946.888889,5090.000000,3330.555556,0.0,0.0,0.0,0.0,0.0,1.111111,0.0
4,202,201814,14378.0,51910.0,37335.0,0.0,0.0,0,0,0,...,1597.555556,5767.777778,4148.333333,0.0,0.0,0.0,0.0,0.0,1.333333,0.0


In [177]:
set_a = set(campaña_agg['IdConsultora'].unique().tolist()) 
set_b = set(campaña_consultora['IdConsultora'].unique().tolist())
inter_ = set_a & set_b

In [504]:
campaña_merge = campaña_consultora.merge(campaña_agg, on=['IdConsultora', 'campana'], how='left')
campaña_merge = campaña_merge.sort_values(by=['IdConsultora', 'campana'])
campaña_merge = campaña_merge.drop(['codigocanalorigen'], axis=1)
campaña_merge.fillna(0, inplace=True)
campaña_merge.head()

,campana,IdConsultora,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,...,ahorro_mean,realvtamncatalogo_mean,realvtamnneto_mean,realanulmnneto_mean,realdevmnneto_mean,realuuanuladas_mean,realuudevueltas_mean,realuufaltantes_mean,realuuvendidas_mean,realvtamnfaltneto_mean
0,201807,125,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,201808,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,201809,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,201810,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,201811,125,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [505]:
campaña_merge['Flag_shift'] = campaña_merge.groupby('IdConsultora')['Flagpasopedido'].shift(-1)
campaña_merge = campaña_merge.select_dtypes(exclude=['object'])

In [506]:
df_val = campaña_merge[campaña_merge['Flag_shift'].isnull()].set_index(['IdConsultora', 'campana'])
df_model = campaña_merge[campaña_merge['Flag_shift'].notna()].set_index(['IdConsultora', 'campana'])

In [564]:
X = df_model.drop(['Flag_shift', 'Flagpasopedido'], axis=1)
y = df_model['Flag_shift']

In [565]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [566]:
X_train.head()

,,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,flagpasopedidoweb,cantidadlogueos,flagdispositivo,...,ahorro_mean,realvtamncatalogo_mean,realvtamnneto_mean,realanulmnneto_mean,realdevmnneto_mean,realuuanuladas_mean,realuudevueltas_mean,realuufaltantes_mean,realuuvendidas_mean,realvtamnfaltneto_mean
IdConsultora,campana,,,,,,,,,,,,,,,,,,,,,
722066,201810,1,0.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,1,...,8065.800,4717.20,3963.7,0.0,0.0,0.0,0.0,0.0,1.70,0.0
467498,201905,1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,6.0,1,...,1648.300,8020.00,5091.2,0.0,0.0,0.0,0.0,0.0,1.00,0.0
657979,201816,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1,...,4002.500,8142.50,5222.8,0.0,0.0,0.0,0.0,0.0,1.15,0.0
95096,201812,1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,1,...,2582.375,7616.25,5154.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0
509760,201814,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


## Random Forest

In [567]:
rforest = RandomForestClassifier(n_estimators=1000, min_samples_leaf=10, n_jobs=-1, random_state=42)
rforest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [568]:
predict_rforest = rforest.predict(X_test)
predict_rforest

array([0., 1., 1., ..., 0., 1., 0.])

In [569]:
accuracy_score(y_test, predict_rforest)

0.8796530105794766

In [570]:
predict_probs = rforest.predict_proba(X_test)[:,1]

In [571]:
auc = roc_auc_score(y_test, predict_probs)
auc

0.9170022808537552

## ExtraTrees

In [573]:
etrees = ExtraTreesClassifier(n_estimators=1000, min_samples_leaf=10, n_jobs=-1, bootstrap=True, random_state=42)
etrees.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=10, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)

In [574]:
predict_etrees = etrees.predict(X_test)

In [575]:
accuracy_score(y_test, predict_etrees)

0.8677529133469749

In [576]:
predict_probs = etrees.predict_proba(X_test)[:, 1]

In [577]:
auc = roc_auc_score(y_test, predict_probs)
auc

0.901012109217527

## Gradient Boosting

In [581]:
gboost = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, min_samples_leaf=10)
gboost.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [582]:
predict_probs = gboost.predict_proba(X_test)[:, 1]

In [583]:
auc = roc_auc_score(y_test, predict_probs)
auc

0.9195065908086555

## Lightgbm

In [596]:
lgbm_class = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.01, n_jobs=-1, random_state=42)
lgbm_class.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [597]:
predict_lgbm = lgbm_class.predict(X_test)

In [598]:
accuracy_score(y_test, predict_lgbm)

0.8822144339472042

In [599]:
predict_probs = lgbm_class.predict_proba(X_test)[:, 1]

In [600]:
auc = roc_auc_score(y_test, predict_probs)
auc

0.9208834961148914

### Crossvalidation

In [594]:
predictions = cross_val_predict(lgbm_class, X, y, cv=5, n_jobs=-1, method='predict_proba', verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.6min remaining:  6.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.7min finished


In [595]:
auc = roc_auc_score(y, predictions[:, 1])
auc

0.911132126797708

## Stacking

In [537]:
def get_oof(clf, x_train, y_train, x_test):
    
    NFOLDS = 5 # set folds for out-of-fold prediction
    kf = KFold(n_splits= NFOLDS, random_state=42)
    
    oof_train = np.zeros((x_train.shape[0],))
    oof_test = np.zeros((x_test.shape[0],))
    oof_test_skf = np.empty((NFOLDS, x_test.shape[0]))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict_proba(x_te)[:,1]
        oof_test_skf[i, :] = clf.predict_proba(x_test)[:,1]

    oof_test = oof_test_skf.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [538]:
rforest1 = RandomForestClassifier(n_estimators=100, min_samples_leaf=10, n_jobs=-1, random_state=42, verbose=5)
rforest2 = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=10, random_state=52, verbose=5)
rforest3 = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=28)
rforest4 = ExtraTreesClassifier(n_estimators=100, min_samples_leaf=10, n_jobs=-1, random_state=2, verbose=5)

In [539]:
rforest1.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100building tree 9 of 100building tree 10 of 100building tree 11 of 100building tree 12 of 100




building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   33.5s


building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  98 out of 100 | elapsed:  1.2min remaining:    1.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=5,
                       warm_start=False)

In [540]:
rforest2.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.1639            3.36m
         2           1.0797            3.26m
         3           1.0121            3.20m
         4           0.9567            3.17m
         5           0.9104            3.14m
         6           0.8713            3.10m
         7           0.8379            3.07m
         8           0.8091            3.03m
         9           0.7845            2.99m
        10           0.7628            2.96m
        11           0.7445            2.93m
        12           0.7280            2.90m
        13           0.7136            2.87m
        14           0.7013            2.84m
        15           0.6899            2.82m
        16           0.6803            2.78m
        17           0.6717            2.75m
        18           0.6638            2.73m
        19           0.6568            2.70m
        20           0.6504            2.67m
        21           0.6450            2.64m
        2

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=52, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=5,
                           warm_start=False)

In [541]:
rforest3.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.1,
                   n_estimators=100, random_state=28)

In [542]:
rforest4.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100building tree 5 of 100building tree 10 of 100building tree 11 of 100



building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   24.6s


building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  98 out of 100 | elapsed:   46.5s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   46.8s finished


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=10, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=2, verbose=5,
                     warm_start=False)

In [543]:
rf_oof_train1, rf_oof_test1 = get_oof(rforest1, X_train, y_train, X_test)
print('-----------------------------------------------------------------------------------------')
rf_oof_train2, rf_oof_test2 = get_oof(rforest2, X_train, y_train, X_test)
print('-----------------------------------------------------------------------------------------')
rf_oof_train3, rf_oof_test3 = get_oof(rforest3, X_train, y_train, X_test)
print('-----------------------------------------------------------------------------------------')
rf_oof_train4, rf_oof_test4 = get_oof(rforest4, X_train, y_train, X_test)
print('-----------------------------------------------------------------------------------------')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100building tree 6 of 100building tree 7 of 100building tree 8 of 100
building tree 9 of 100




building tree 10 of 100building tree 11 of 100



building tree 12 of 100

building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   25.4s


building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  98 out of 100 | elapsed:   51.3s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   51.4s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.5s
[Parallel(n_jobs=12)]: Done  98 out of 100 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.6s
[Parallel(n_jobs=12)]: Done  98 out of 100 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   18.0s


building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done  98 out of 100 | elapsed:   35.0s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   35.2s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done  98 out of 100 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.4s


-----------------------------------------------------------------------------------------


[Parallel(n_jobs=12)]: Done  98 out of 100 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.8s finished


In [544]:
base_predictions_train = pd.DataFrame( 
    {'RF1': rf_oof_train1.ravel(),
     'RF2': rf_oof_train2.ravel(),
     'RF3': rf_oof_train3.ravel(),
     'RF4': rf_oof_train4.ravel()
    })
base_predictions_train['idconsultora'] = X_train.index.tolist()
base_predictions_train.set_index('idconsultora', inplace=True)
base_predictions_train.head()

,RF1,RF2,RF3,RF4
idconsultora,,,,
"(722066, 201810)",0.652285,0.687269,0.526972,0.840727
"(467498, 201905)",0.958880,0.960667,0.568032,0.942978
"(657979, 201816)",0.958809,0.952552,0.567174,0.956855
"(95096, 201812)",0.898963,0.909820,0.557553,0.885310
"(509760, 201814)",0.589654,0.538372,0.505801,0.500559


In [545]:
base_predictions_test = pd.DataFrame( 
    {'RF1': rf_oof_test1.ravel(),
     'RF2': rf_oof_test2.ravel(),
     'RF3': rf_oof_test3.ravel(),
     'RF4': rf_oof_test4.ravel()
    })
base_predictions_test['idconsultora'] = X_test.index.tolist()
base_predictions_test.set_index('idconsultora', inplace=True)
base_predictions_test.head()

,RF1,RF2,RF3,RF4
idconsultora,,,,
"(749123, 201810)",0.027021,0.002501,0.194601,0.079835
"(375556, 201904)",0.968198,0.965670,0.570141,0.950405
"(669777, 201901)",0.924331,0.911433,0.556157,0.909877
"(781909, 201901)",0.026977,0.002501,0.194601,0.069461
"(396589, 201818)",0.866373,0.841835,0.550002,0.836975


In [558]:
lgbm_stack = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.001, n_jobs=-1, random_state=42)

In [559]:
lgbm_stack.fit(base_predictions_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.001, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [560]:
probs = lgbm_stack.predict_proba(base_predictions_test)[:, 1]

In [561]:
auc = roc_auc_score(y_test, probs)
auc

0.9201425776244518

# Submition

In [530]:
df_val.head()

,,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,flagpasopedidoweb,cantidadlogueos,...,realvtamncatalogo_mean,realvtamnneto_mean,realanulmnneto_mean,realdevmnneto_mean,realuuanuladas_mean,realuudevueltas_mean,realuufaltantes_mean,realuuvendidas_mean,realvtamnfaltneto_mean,Flag_shift
IdConsultora,campana,,,,,,,,,,,,,,,,,,,,,
125,201812,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,NaN
202,201906,1,1,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,...,6965.000000,4977.375000,0.0,0.000000,0.0,0.000000,0.0,1.750000,0.0,NaN
243,201906,1,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,5.0,...,4770.288462,3162.134615,0.0,0.000000,0.0,0.000000,0.0,1.269231,0.0,NaN
274,201906,1,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,4.0,...,6362.318841,3715.811594,0.0,0.000000,0.0,0.000000,0.0,1.275362,0.0,NaN
293,201906,1,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,11.0,...,8295.327273,5768.236364,0.0,335.054545,0.0,0.154545,0.0,2.200000,0.0,NaN


In [531]:
predict_bel.head()

,idconsultora,flagpasopedido
0,175391,NaN
1,336826,NaN
2,667498,NaN
3,386050,NaN
4,504579,NaN


In [532]:
df_sub_tmp = df_val.loc[predict_bel['idconsultora'].tolist()].drop('Flag_shift', axis=1)

## Sub 1

In [533]:
df_sub_tmp['Flag_shift'] = lgbm_class.predict_proba(df_sub_tmp)[:, 1]
df_sub_tmp.head()

,,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,flagpasopedidoweb,cantidadlogueos,...,realvtamncatalogo_mean,realvtamnneto_mean,realanulmnneto_mean,realdevmnneto_mean,realuuanuladas_mean,realuudevueltas_mean,realuufaltantes_mean,realuuvendidas_mean,realvtamnfaltneto_mean,Flag_shift
IdConsultora,campana,,,,,,,,,,,,,,,,,,,,,
202,201906,1,1,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,...,6965.000000,4977.375000,0.0,0.000000,0.0,0.000000,0.000000,1.750000,0.000000,0.789074
243,201906,1,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,5.0,...,4770.288462,3162.134615,0.0,0.000000,0.0,0.000000,0.000000,1.269231,0.000000,0.985664
274,201906,1,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,4.0,...,6362.318841,3715.811594,0.0,0.000000,0.0,0.000000,0.000000,1.275362,0.000000,0.983723
293,201906,1,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,11.0,...,8295.327273,5768.236364,0.0,335.054545,0.0,0.154545,0.000000,2.200000,0.000000,0.980328
431,201906,1,1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,...,4908.333333,3572.666667,0.0,0.000000,0.0,0.000000,0.083333,1.166667,209.583333,0.963682


In [534]:
df_sub = df_sub_tmp.reset_index()[['IdConsultora', 'Flag_shift']]
df_sub = df_sub.set_index('IdConsultora').loc[predict_bel['idconsultora'].tolist()].reset_index()
df_sub.rename(columns={'IdConsultora': 'idconsultora', 'Flag_shift': 'flagpasopedido'}, inplace=True)
df_sub.head()

,idconsultora,flagpasopedido
0,175391,0.970491
1,336826,0.963737
2,667498,0.000030
3,386050,0.924478
4,504579,0.963879


In [535]:
df_sub.shape, predict_bel.shape

((120895, 2), (120895, 2))

In [536]:
df_sub.to_csv('watermelon3.csv', encoding='utf-8', index=None)

## Sub 2

In [437]:
base_val = pd.DataFrame(
    {'RF1': rforest1.predict_proba(df_sub_tmp)[:, 1],
     'RF2': rforest2.predict_proba(df_sub_tmp)[:, 1],
     'RF3': rforest3.predict_proba(df_sub_tmp)[:, 1],
     'RF4': rforest4.predict_proba(df_sub_tmp)[:, 1],
})

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done  98 out of 100 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done  98 out of 100 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done  98 out of 100 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  4

In [438]:
df_sub_tmp['flagpasopedido'] = lgbm_stack.predict_proba(base_val)[:, 1]

In [439]:
df_sub_tmp.head()

,,Flagpasopedido,flagactiva,flagpasopedidocuidadopersonal,flagpasopedidomaquillaje,flagpasopedidotratamientocorporal,flagpasopedidotratamientofacial,flagpedidoanulado,flagpasopedidofragancias,flagpasopedidoweb,cantidadlogueos,...,realvtamncatalogo,realvtamnneto,realanulmnneto,realdevmnneto,realuuanuladas,realuudevueltas,realuufaltantes,realuuvendidas,realvtamnfaltneto,flagpasopedido
IdConsultora,campana,,,,,,,,,,,,,,,,,,,,,
202,201906,1,1,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,...,55720.0,39819.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.850794
243,201906,1,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,5.0,...,248055.0,164431.0,0.0,0.0,0.0,0.0,0.0,66.0,0.0,0.963110
274,201906,1,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,4.0,...,439000.0,256391.0,0.0,0.0,0.0,0.0,0.0,88.0,0.0,0.969671
293,201906,1,1,1.0,1.0,1.0,1.0,0.0,1.0,1.0,11.0,...,912486.0,634506.0,0.0,36856.0,0.0,17.0,0.0,242.0,0.0,0.969671
431,201906,1,1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,...,58900.0,42872.0,0.0,0.0,0.0,0.0,1.0,14.0,2515.0,0.843239


In [440]:
df_sub2 = df_sub_tmp.reset_index()[['IdConsultora', 'flagpasopedido']]
df_sub2 = df_sub2.set_index('IdConsultora').loc[predict_bel['idconsultora'].tolist()].reset_index()
df_sub2.rename(columns={'IdConsultora': 'idconsultora'}, inplace=True)
df_sub2.head()

,idconsultora,flagpasopedido
0,175391,0.892917
1,336826,0.900841
2,667498,0.103959
3,386050,0.789180
4,504579,0.864189


In [441]:
df_sub2.to_csv('watermelon2.csv', encoding='utf-8', index=None)

## Sub 3